In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

from utils import get_completion

model_id = "kreimben/CodeMind-gemma-2b"

tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side="left")
model = AutoModelForCausalLM.from_pretrained(model_id)

config.json:   0%|          | 0.00/699 [00:00<?, ?B/s]

C:\Users\aksid\PycharmProjects\CodeMind\venv\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\aksid\.cache\huggingface\hub\models--kreimben--CodeMind-gemma-2b. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [2]:
model = model.to('cuda')

In [3]:
%%time

text = '''
class Solution:
    def topKFrequent(self, nums: List[int], k: int) -> List[int]:
        nums.sort()
        d = defaultdict()

        for num in nums:
            d[num] += 1
'''

res = get_completion(
    text,
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
C:\Users\aksid\PycharmProjects\CodeMind\venv\lib\site-packages\transformers\models\gemma\modeling_gemma.py:573: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


CPU times: total: 8.14 s
Wall time: 25.8 s


In [4]:
answer = res.split('model')[-1]

In [5]:
from IPython.display import display, Markdown

# answer = answer.replace('\\\\', '\\')

display(Markdown(answer))

The provided code is a Python implementation of the `topKFrequent()` method. It takes two arguments:

- `nums`: A list of integers.
- `k`: An integer representing the maximum number of elements to return.

**Code:**
```python
class Solution:
    def topKFrequent(self, nums: List[int], k: int) -> List[int]:
        nums.sort()
        d = defaultdict()

        for num in nums:
            d[num] += 1

        # Count the frequency of each element
        freq = {}
        for num, count in d.items():
            freq[count] = num

        # Get the top k most frequent elements
        top_k = []
        for i, num in enumerate(freq.items()):
            top_k.append(num)
            if len(top_k) > k:
                top_k.pop(0)

        return top_k
```

**Explanation:**

- The code first sorts the `nums` list in ascending order.
- It then uses a `defaultdict` named `d` to store the frequency of each element in the sorted list.
- The code iterates through the sorted list and increments the frequency of each element in `d`.
- It then creates a dictionary named `freq` that maps the frequency of each element to the element itself.
- The code iterates through the `d` dictionary and counts the frequency of each element.
- It then uses a loop to iterate through the `freq` dictionary and add the top `k` elements to the `top_k` list.
- The code removes the first element from the `top_k` list if the length of `top_k` exceeds `k`.
- Finally, the code returns the `top_k` list.

**Time Complexity:** O(n log n)
**Space Complexity:** O(n)

**Note:**

- The time complexity of the code is O(n log n) because it uses a merge sort to sort the `nums` list.
- The space complexity of the code is O(n) because it uses a dictionary `d` to store the frequency of each element. The provided code is a Python implementation of the `topKFrequent()` method. It takes two arguments:

- `nums`: A list of integers.
- `k`: An integer representing the maximum number of elements to return.

**Code:**
```python
class Solution:
    def topKFrequent(self, nums: List[int], k: int) -> List[int]:
        nums.sort()
        d = defaultdict()

        for num in nums:
            d[num] += 1

        # Count the frequency of each element
        freq = {}
        for num, count in d.items():
            freq[count] = num

        # Get the top k most frequent elements
        top_k = []
        for i, num in enumerate(freq.items()):
            top_k.append(num)
            if len(top_k) > k:
                top_k.pop(0)

        return top_k
```

**Explanation:**

- The code first sorts the `nums` list in ascending order.
- It then uses a `defaultdict` named `d` to store the frequency of each element in the sorted list.
- The code iterates through the sorted list and increments the frequency of each element in `d`.
- It then creates a dictionary named `freq` that maps the frequency of each element to the element itself.
- The code iterates through the `d` dictionary and adds the top `k` elements to the `top_k` list.
- The code removes the first element from the `top_k` list if the length of `top_k` exceeds `k`.
- Finally, the code returns the `top_k` list.

**Time Complexity:** O(n log n)
**Space Complexity:** O(n)

**Note:**

- The time complexity of the code is O(n log n) because it uses a merge sort to sort the `nums` list.
- The space complexity of the code is O(n) because it uses a dictionary `d` to store the frequency of each element. The provided code is a Python implementation of the `topKFrequent()` method. It takes two arguments:

- `nums`: A list of integers.
- `k`: An integer representing the maximum number of elements to return.

**Code:**
```python
class Solution:
    def topKFrequent(self, nums: List[int], k: int) -> List[